In [1]:
import pandas as pd

In [2]:
precomputed = ['Ecosystem Vitality', 'Environmental Health', 'Air Quality', 'Sanitation & Drinking Water']

In [3]:
df = pd.read_csv('../data/dataframes/epi_cleaned.csv')

In [4]:
df.drop(columns='Unnamed: 0', inplace=True)

In [5]:
epi_2020 = df[df['year'] == 2020]

In [6]:
epi_2020.reset_index(inplace=True)

In [7]:
index_list_high = epi_2020['Environmental Performance Index'].sort_values(ascending=False).head(10).index

In [8]:
epi_2020.loc[158, 'country_name']

'Denmark'

In [9]:
top_country_list = []
for index in index_list_high:
    top_country_list.append(epi_2020.loc[index, 'country_name'])

In [10]:
top_country_list

['Denmark',
 'Luxembourg',
 'Switzerland',
 'United Kingdom',
 'France',
 'Austria',
 'Finland',
 'Sweden',
 'Norway',
 'Germany']

In [11]:
index_list_low = epi_2020['Environmental Performance Index'].sort_values(ascending=False).tail(10).index

In [12]:
index_list_low

Int64Index([141, 44, 16, 19, 170, 118, 24, 96, 52, 103], dtype='int64')

In [13]:
bot_country_list = []
for index in index_list_low:
    bot_country_list.append(epi_2020.loc[index, 'country_name'])

In [14]:
bot_country_list

['Burundi',
 'Chad',
 'Solomon Islands',
 'Madagascar',
 'Guinea',
 "Cote d'Ivoire",
 'Sierra Leone',
 'Afghanistan',
 'Myanmar',
 'Liberia']